In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import h5py
from torch.autograd import Variable
import os
from torch.utils.data import Dataset, DataLoader
import statistics

In [6]:
import sys
print(sys.executable)

/home/oussema/anaconda3/envs/open-mmlab/bin/python


In [7]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [8]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [105]:
#load data
def load_file(file, flabel,subseq ):
    
    
    print(file)
    f = open(file, 'r')
    Lines = f.readlines()
    t = 1
    data = []
    for line in Lines:
        t += 1
        line = line.split()[0:75] #75=25*3

        data += line
    
    fl = open(flabel, 'r')
    label = fl.readlines()[0]
    
    
    #data
    samples =len(label)
    data_len = samples * subseq * 75
    data = data[0:data_len]
    data = [float(i) for i in data] 
    data = np.array(data)
    data = data.reshape(samples,subseq,75)
    
    #eliminate joints
    #jts = [1,2,5,6,7,9,10,11,12,13,14,16,17,18,20,22,24]
    #jts_x = [x*3 for x in jts]
    #jts_y = [x+1 for x in jts_x]
    #jts_z = [x+2 for x in jts_x]
    #jts_x = jts_x + jts_y + jts_z
    #jts_x.sort()
    #data = np.delete(data, jts_x, axis=2)
    

    #label
    label = [int(x) for x in (label)]     
    label = np.asarray(label)
    #print (data.shape, label.shape)

    f.close()
    
    return data,label



In [86]:
def load_dataset(data_path, flabel_path,subseq):
    data = []
    label = []
    
    for filename in os.listdir(data_path):
        data1, label1 = load_file(os.path.join(data_path,filename),os.path.join(flabel_path,filename),subseq)
        data.append(torch.tensor(data1).float())
        label.append(torch.tensor(label1))
        
    data= torch.cat(data, 0)
    data = data.permute(0,2,1)
    
    
    label= torch.cat(label, 0)
    
    print (data.shape)
    print (label.shape)
    
    return data,label

In [87]:
class MyDataset(Dataset):
    def __init__(self, data_path, flabel_path, subseq):
        #data loading
        data, label = load_dataset(data_path, flabel_path, subseq)
        self.x = data
        self.y = label
        self.n_samples = data.shape[0]
        
        
    def __getitem__(self, index):
        #dataset[0]
        return self.x[index], self.y [index]
        
    def __len__(self):
        #len dataset
        return self.n_samples
        
        

In [106]:
datapku_train = MyDataset(data_path= '/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train', flabel_path='/home/oussema/code/st-gcn/pku/pku_action_detect/train',subseq = 30)
dataloader_train = DataLoader(dataset=datapku_train, batch_size = 200)

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0226-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0348-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0073-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0214-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0212-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0359-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0133-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0114-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0227-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0356-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0211-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0266-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0221-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/033

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0200-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0218-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0173-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0245-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0132-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0217-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0172-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0161-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0081-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0160-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0356-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0036-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0255-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/022

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0256-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0120-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0251-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0107-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0075-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0013-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0261-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0118-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0017-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0143-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0020-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0085-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0021-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/015

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0146-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0196-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0268-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0101-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0116-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0274-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0070-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0168-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0213-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0018-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0085-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0067-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0103-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/011

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0199-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0082-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0047-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0131-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0362-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0060-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0289-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0350-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0074-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0033-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0120-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0279-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0183-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/005

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0352-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0337-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0268-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0159-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0239-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0031-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0288-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0056-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0358-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0228-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0068-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0162-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0062-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/000

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0127-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0091-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0224-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0263-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0273-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0251-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0191-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0061-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0082-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0137-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0336-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0264-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0180-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/004

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0219-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0359-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0045-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0127-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0282-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0195-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0172-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0351-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0044-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0132-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0015-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0280-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0015-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/015

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0219-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0111-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0142-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0205-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0255-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0271-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0207-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0017-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0343-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0125-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0261-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train/0364-M.txt
torch.Size([149869, 75, 30])
torch.Size([149869])


In [107]:
datapku_test = MyDataset(data_path= '/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test', flabel_path='/home/oussema/code/st-gcn/pku/pku_action_detect/test',subseq = 30)
dataloader_test = DataLoader(dataset=datapku_test, batch_size = 200)

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0309-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0292-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0293-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0309-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0322-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0295-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0327-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0321-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0320-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0299-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0323-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0295-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0318-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0296-R.txt
/home/

/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0330-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0329-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0315-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0305-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0333-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0332-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0311-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0310-L.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0301-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0291-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0314-M.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0294-R.txt
/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test/0300-R.txt
torch.Size([20375, 75, 30])
torch.Size([20375])


In [81]:
#afficher data
dataiter = iter(dataloader_train)
data = dataiter.next()
x,y = data

In [83]:
x[0,:,0]

tensor([-0.0464, -0.0772,  2.4141, -0.0587,  0.7254,  2.3942, -0.2211,  0.4327,
         2.2604,  0.0468,  0.4288,  2.4593, -0.1909, -0.8657,  2.3098, -0.0057,
        -0.8638,  2.4277, -0.2019, -0.1328,  2.1976,  0.1000, -0.1562,  2.4467])

In [108]:
KERNEL_SIZE = 3
PAD = 1
LEARNING_RATE = 0.1
#conv1d input [batch_size, in_channels, len]
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            # Layer 1: Convolution - ReLU - Max pooling
            nn.Conv1d(in_channels=75, out_channels=128, kernel_size=KERNEL_SIZE, padding=PAD),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Layer 2: Convolution - ReLU - Max pooling
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=KERNEL_SIZE, padding=PAD),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            # Layer 3: Convolution - ReLU - Max pooling
            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=KERNEL_SIZE, padding=PAD),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        # Layer 5-6-7: Fully connected linear layers
        self.fc1 = nn.Linear(512*3, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 3)
        
    def forward(self, x):
        temp = self.conv(x)
        temp = temp.view(-1,512*3)
        temp = self.fc1(temp)
        temp = self.fc2(temp)
        temp = self.fc3(temp)
        #temp = torch.softmax(temp, dim=0)
        return temp


            

In [109]:
cnn = Classifier()
if cuda_available:
    cnn = cnn.cuda()
    
optimizer = torch.optim.SGD(cnn.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss() 

pytorch_total_params = sum(p.numel() for p in cnn.parameters())
print('Number of parameters in the model: %d' % pytorch_total_params)
print(cnn)

Number of parameters in the model: 931331
Classifier(
  (conv): Sequential(
    (0): Conv1d(75, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1536, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
)


In [110]:
#training
epochs = 50
for epoch in range (epochs):
    correct = 0
    train_losses = []
    
    for batch_idx, (data,label) in enumerate(dataloader_train):
        
        if cuda_available:
                data, label = data.cuda(), label.cuda()
                
        optimizer.zero_grad()
        output = cnn(data.float())
        loss = criterion (output, label)
        pred = output.data.max(1)[1]
        correct += pred.eq(label).sum().item()
        train_losses.append(loss.item())
        
        loss.backward()
        optimizer.step()
        
    train_acc = 100*correct/len(datapku_train)
    print(epoch," : ",train_acc)
    print(statistics.mean(train_losses))
    
    


0  :  56.73488179676918
0.6904957330226899
1  :  58.836050150464736
0.6764714238643647
2  :  59.320473213272926
0.6659038407802582
3  :  65.63398701532672
0.6174594844182333
4  :  69.55207547925188
0.5838897982438406
5  :  76.668290306868
0.5029273314476013
6  :  78.60664980749854
0.4759161809682846
7  :  80.5623577924721
0.4460242840051651
8  :  81.40375928310725
0.43386409056186676
9  :  81.95757628328741
0.42627007126808164
10  :  82.24716252193582
0.4219333798885345
11  :  82.33924293883325
0.42216032872597375
12  :  82.75694106186069
0.4134181282718976
13  :  82.96912637036345
0.4103092799584071
14  :  82.89105819081998
0.41113434950510663
15  :  83.16930118970568
0.4065005485614141
16  :  83.39950223194924
0.40255314071973164
17  :  83.40951097291635
0.400541798055172
18  :  83.43686819822645
0.40233605237801867
19  :  83.7024334585538
0.39552045295635857
20  :  83.65105525492264
0.39515093898773196
21  :  83.77783264050605
0.3937864441672961
22  :  83.92129126103464
0.3903206396

In [111]:
# run a test loop
test_loss = 0
correct = 0


with torch.no_grad():  
    for batch_idx, (data,label) in enumerate(dataloader_test):
        if cuda_available:
                data, label = data.cuda(), label.cuda()
        net_out = cnn(data)
        # sum up batch loss
        test_loss += criterion(net_out, label).data
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(label.data).sum().item()
        
    test_acc = 100*correct/len(datapku_test)
    print (test_acc)


83.99509202453987


In [70]:
l=[[[1,1,1],[2,2,2]]]

In [71]:
l.shape

AttributeError: 'list' object has no attribute 'shape'